# Projet 6 : Catégorisez automatiquement des questions
# <u>B. Topic Modeling</u> <br/>

# Le contexte

Afin d'aider les utilisateurs de Stack Overflow dans leur soumission de question, nous devons mettre en place un système de suggestion de tags. Pour celà nous allons nous baser sur les techniques de machine learning capable en fonction du texte saisi par l'utilisateur de déterminer des tags pertinents.

Dans ce notebook nous allons essayer des approches non supervisées.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from time import time

from bs4 import BeautifulSoup
import unicodedata
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.toktok import ToktokTokenizer
from contractions import CONTRACTION_MAP


from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

%matplotlib inline

# 1. Chargement des données pré-traitées

Nous allons charger les données qui ont été préalablement nettoyées et traitées.

In [2]:
df = pd.read_csv('cleaned_data.csv')
#replace NaN by empty string
df = df.replace(np.nan, '', regex=True)

In [3]:
df.shape

(64432, 7)

In [4]:
df.head()

,TITLE,BODY,SCORE,TAGS,TITLE_P,BODY_P,TAGS_P
0,Java generics variable <T> value,<p>At the moment I am using the following code...,6,<java><generics>,java gener variabl valu,moment use follow code filter jpa reduc block ...,"['java', 'generics']"
1,How a value typed variable is copied when it i...,<blockquote>\n <p>Swift's string type is a va...,6,<swift><function><value-type>,valu type variabl copi pass function hold copi,swift string type valu type creat new string v...,"['swift', 'function', 'value-type']"
2,Error while waiting for device: The emulator p...,<p>I am a freshman for the development of the ...,6,<android><android-studio><android-emulator><avd>,error wait devic emul process avd kill,freshman develop andriod suffer odd question r...,"['android', 'android-studio', 'android-emulato..."
3,gulp-inject not working with gulp-watch,<p>I am using gulp-inject to auto add SASS imp...,10,<javascript><node.js><npm><gulp><gulp-watch>,gulp inject work gulp watch,use gulp inject auto add sass import newli cre...,"['javascript', 'node.js', 'npm', 'gulp', 'gulp..."
4,React - Call function on props change,<p>My TranslationDetail component is passed an...,12,<reactjs><react-router>,react call function prop chang,translationdetail compon pass id upon open bas...,"['reactjs', 'react-router']"


In [5]:
df.dtypes

TITLE      object
BODY       object
SCORE       int64
TAGS       object
TITLE_P    object
BODY_P     object
TAGS_P     object
dtype: object

La colonne TAGS_P est interprétée comme un chaîne de caractères. On va la forcer à être vue comme une list, ce qui sera plus simple pour les traitements.

In [6]:
from ast import literal_eval
df['TAGS_P'] = df['TAGS_P'].apply(literal_eval)

# 2. Préparation des données

## 2.1 Echantillonage

Nous avons plus de 64 000 posts. Nous allons nous concentrer sur un échantillon de 20 000 posts.

In [7]:
df_sample = df.sample(20000)

In [8]:
df_sample.shape

(20000, 7)

Séparons nos données pour garder 5000 posts pour la validation de nos modèles. <br/>
Nous utiliserons les 15 000 autres lignes pour faire l'apprentissage.

In [9]:
df_learn = df_sample.iloc[5000:, :].copy()
df_validation = df_sample.iloc[:5000, :].copy()

In [10]:
display(df_learn.shape)
display(df_validation.shape)

(15000, 7)

(5000, 7)

** Analysons le nombre de mots que contient ce corpus (entrainement).**

In [11]:
my_counter = Counter()
for sentence in df_learn['TITLE_P']:
    my_counter.update(sentence.split())
for sentence in df_learn['BODY_P']:
    my_counter.update(sentence.split())
len(my_counter)

90261

=> Notre corpus est composé de près de **90 000 mots.**

## 2.2 Filtre sur les tags les plus fréquents

Regardons déjà dans un premier temps le nombre d'occurences par tag dans notre dataset complet.

In [12]:
counts = Counter()
for tags_list in df['TAGS_P']:
    counts.update(tags_list)
tags_df = pd.DataFrame.from_dict(counts, orient='index')
tags_df.reset_index(drop = False, inplace = True)
tags_df= tags_df.rename(columns={'index':'tag', 0:'count'})

La structures **tags_df** contient pour chacun des tags son occurence. <br/>
Gardons que les tags qui sont présents dans au moins 10 documents.

In [13]:
len(tags_df[tags_df['count'] > 10])

2114

Nous nous retrouvons donc avec un peu plus de 2100 tags (14000 tags dans le dataset original).

- Filtrons maintenant notre sample en ne gardant que les posts contenant les tags les plus fréquents.

In [14]:
frequent_tags = tags_df[tags_df['count'] > 10]['tag'].tolist()

**frequent_tags** est la structure contenant les tags les plus présents.

In [15]:
df_learn['TAGS_P'] = df_learn['TAGS_P'].apply(lambda x: [w for w in x if w in frequent_tags] )
# On supprime les lignes qui n'ont plus de tags associés (car aucun n'est présent dans la liste frequent_tags)
df_learn = df_learn[df_learn.astype(str)['TAGS_P'] != '[]']

In [16]:
df_learn.shape

(14845, 7)

Notre dataset contient maintenant uniquement les posts avec tags fréquents.

## 2.3 Découpage en jeu d'entrainement et test

Découpons nos données en jeu d'entrainement et jeu de test pour nos algorithmes d'apprentissage.<br/>
Nous gardons les colonnes titre et body.

In [17]:
X = df_learn[['TITLE', 'BODY','TITLE_P', 'BODY_P']]
Y = df_learn[['TAGS_P']]

- 70% des données pour entrainement et 30% pour les tests.

In [18]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X,Y,test_size = 0.3,random_state = 0, shuffle = True)

In [19]:
print("train", x_train.shape)
print("test ",x_test.shape)

train (10391, 4)
test  (4454, 4)


# 3. Préparation pour le topic modeling

## 3.1 Matrice Topics / Words

Ecrivons une méthode réutilisable qui affiche pour un modèle de topic modeling donné, la matrice Topics/Words

In [21]:
def getTopicsWordsMatrix(model, features_name) :
    # liste des topics
    topicnames = ["Topic" + str(i) for i in range(model.n_components)]

    # Topic-Keyword Matrice
    df_topic_keywords = pd.DataFrame(model.components_)
    df_topic_keywords.columns = features_name
    df_topic_keywords.index = topicnames

    return df_topic_keywords

In [22]:
'''
Méthode qui retourne pour chacun des topics le top "n_words" mots
@param feature_names : les features de nos données
@param tp_model : le model utilisé pour le topic modeling
@param n_words : top nombre de mots
'''
def showTopicsTopWords(tp_model, feature_names, n_words=10):
    keywords = np.array(feature_names)
    topic_keywords = []
    for topic_weights in tp_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    display(df_topic_keywords)

## 3.2 Matrice Docs / Topics

Méthode qui retourne la matrice documents / topics suite à l'apprentissage de type topic modeling.

In [23]:
'''
    Retourne la matrice qui indique la pertinence des topics pour chacun des documents de notre jeu de données.
'''
def getDocsTopicsMatrix(df_docs, model, model_output):
    # id des posts
    docnames = ["Doc" + str(i) for i in range(len(df_docs))]
    # id des topics
    topicnames = ["Topic" + str(i) for i in range(model.n_components)]
    # On crée un dataframe
    df_document_topic = pd.DataFrame(np.round(model_output, 4), columns=topicnames, index=docnames)
    # On rajoute une colonne contenant le numéro du topic prédominant
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['Dominant_Topic'] = dominant_topic
    return df_document_topic

In [24]:
'''
Affiche pour chacun des topics, le top n documents qui sont les plus pertinent pour ce topic.
On affiche pour chacun des documents le titre mais également les tags associés.
@topics_words : word_to_topics result
@model_output : topics_to_docs model output
'''
def showTopicsTopDocs(topic_word_dist, model_output, feature_names, documents, targets, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(topic_word_dist):
        print ("Topic %d : " % (topic_idx) + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( model_output[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print ("Doc",doc_index," Title:", documents.iloc[doc_index].TITLE[0:60],
                   "- Tags:", targets.iloc[doc_index].TAGS_P)
        print("")

## 3.3 Prédiction des tags

Pour nous aider à faire nos prédictions, nous allons construire une matrice tags par topic.<br/>
Pour la construire, nous allons :
- Pour chaque tag_i et chaque topic_j
    - Sommer la probabilité d’appartenance au topic_j des documents contenant le tag_i. 
    - Nous allons normaliser cette somme en faisant une moyenne sur le nombre de documents qui contiennent le tag_i

|         | tag_1 | tag_2 | ... |tag_m | 
|---------|-------|-------|-----|------|
| **topic_1**  |  xxx  | xxx   | ... | xxx  | 
|**topic_2** |  xxx  | xxx   | ... | xxx  | 
|...      |  xxx  | xxx   | ... | xxx  |
|**topic_n**      |  xxx  | xxx   | ... | xxx  |



- Commençons par écrire une méthode qui donne la liste des indexes des documents qui contiennent un tag donné.

In [25]:
'''
Retourne un dictionnaire contenant pour chacun des tags (parmi les plus fréquents)
les positions des documents contenant ce tag.
Attention il s'agit de la position et non de l'index.
'''
def getDocumentsByTags(df_docs_tags) :
    documents_by_tags = {}
    for tag in frequent_tags :
        doc_indexes = df_docs_tags[df_docs_tags['TAGS_P'].apply(lambda x: True  if tag in x else False)].index.values
        docs_pos = [(lambda x: df_docs_tags.index.get_loc(x))(x) for x in doc_indexes]
        documents_by_tags[tag] = docs_pos
    return documents_by_tags

- Construisons maintenant notre matrice Tags / Topics

In [93]:
'''
@topics_modeling_algo : le modèle.
@ df_docs_tags : les données contenant les tags associés à chacun des documents.
'''
def buildTagByTopicsMatrix(tm_algo, tm_output, df_docs_tags) :
    documents_by_tags = getDocumentsByTags(df_docs_tags)
    tag_by_topics_ = np.zeros([tm_algo.n_components,len(frequent_tags)])
    for topic_idx in range(tm_algo.n_components):
        for tag_idx, tag in enumerate(frequent_tags):
            docs_pos = documents_by_tags[tag]
            topic_score = 0
            for d in docs_pos :
                topic_score += tm_output[d][topic_idx]
                if topic_score > 0 :
                    #tag_by_topics_[topic_idx][tag_idx]=topic_score/len(docs_pos)
                    tag_by_topics_[topic_idx][tag_idx]=topic_score
    return tag_by_topics_

### Tags prédiction

- Méthode qui va chercher dans la matrice tags/topics, les tags les plus pertinent pour un post.<br/>
Elle va d'abord déterminer les probabilité de chaque topic pour le post puis rechercher les tags les plus pertinents.

In [27]:
'''
Retourne la liste de n tags les plus pertinents pour le post donné.
Se base pour celà de la matrice tags par topics.
'''
def getTagsPrediction(post_text, num_tags, tag_topics_matrix, vectorizer, tm_model ) :
    # On  vectorize le texte 
    text_vectorized = vectorizer.transform([post_text])
    topic_probability_scores = tm_model.transform(text_vectorized)
    
    # On multiple les topics prédits par le modèle par notre matrice tags/topics. 
    #On aura ainsi un score pour chacun des tags
    tags_result = topic_probability_scores.dot(tag_topics_matrix)
    # Récupérons maintenant les tags qui ont le meilleur score.
    best_tags_indices = np.argsort(-tags_result[0])[:num_tags]
    best_tags = [frequent_tags[index] for index in best_tags_indices]
    return best_tags

- Prédiction pour une série de posts.

In [28]:
'''
Retourne un dataframe contenant les prédictions de tags pour les posts contenant dans posts_df
@ post_df : le dataframe contenant les posts (title_p / body_p)
@ num_tag : le nombre de tags à prédire
@ t_by_topics : la matrice tags/topics utilisée pour la prédiction
@ vectorizer : convertisseur à utiliser pour transformer texte en matrice de tokens et d'occurences.
@ tm_model : le modèle de topic modeling à utiliser
'''
def getPostsTagsPrediction(posts_df, num_tag, tags_topics_matrix, vectorizer, tm_model, tm_output):
    y_predicted=posts_df.apply(lambda row: getTagsPrediction(row['TITLE_P'] + ' ' + row['BODY_P'], 
                                                             num_tag, tags_topics_matrix, vectorizer, tm_model),axis=1)
    y_predicted_df = y_predicted.to_frame()
    y_predicted_df.columns = ['TAGS_P']
    return y_predicted_df

### Prédiction accuracy

In [29]:
'''
Méthode permettant d'évaluer la qualité des prédictions en comparant les tags prédits aux tags réels.
calcule pour chaque post, le rapport entre le nombre de tags correctement prédits sur le nombre de tags réels.
retourne la moyenne de ces rapports.
'''
def predictionAccuracy(y_predicted, y_true) :
    tags_found=[]
    for index, row in y_predicted.iterrows():
        number_tags_found = 0
        for t in row['TAGS_P'] :
            if t in y_true.loc[index]['TAGS_P'] :
                number_tags_found +=1
        tags_found.append(number_tags_found/len(y_true.loc[index]['TAGS_P']))
    print("Prediction accuracy: {:.2f} % ".format(100*np.mean(tags_found)))

# 4. Topic Modeling LDA

Essayons LDA (Allocation de Dirichlet Latente) pour modéliser les sujets.

Pour pouvoir appliquer nos algorithmes de machine learning à nos données textuelles, il faut en extraire les features et représenter notre texte dans un modèle "mathématique".
Pour celà nous allons utiliser la modélisation **Bag of Words** qui va donner une représentation sous forme de matrice de nos données.

## 4.1 Tuning du modèle avec recherche sur grille

- min_df : le mot doit être présent au moins dans min_df documents
- max_df : si le mot est présent dans plus de max_df des documents, il ne nous aidera pas à trouver différencier les documents. On ne le tient donc pas en compte
- num_topics : nombre de topics du modèle.

### Apprentissage

In [30]:
t0 = time()
Kfold = 5
# CHANGE
search_params = { 
              'vect__min_df': [5, 10, 15],
              'vect__max_df': [0.75, 0.85, 0.95],
              'model__n_components' : [10, 20, 30],
              'model__learning_method':['batch'], 
              'model__max_iter':[5]
             }
# Init the Model
lda = LatentDirichletAllocation()
pipeline = Pipeline([('vect', CountVectorizer()), ('model', lda)])

# Init Grid Search Class
lda_model_grid = GridSearchCV(pipeline, param_grid=search_params)

# On concatène le titre et le body
x_train_text = x_train['TITLE_P'] + ' ' + x_train['BODY_P']
# Do the Grid Search
lda_model_grid.fit(x_train_text)

print("Best params :", lda_model_grid.best_params_)
print("done in %0.3fs." % (time() - t0))

Best params : {'model__learning_method': 'batch', 'model__max_iter': 5, 'model__n_components': 20, 'vect__max_df': 0.95, 'vect__min_df': 5}
done in 172.797s.


- Le LDA avec meilleur performance

In [31]:
best_lda_model = lda_model_grid.best_estimator_.steps[1][1]

- On récupère le vectorizer

In [32]:
tf_vectorizer = lda_model_grid.best_estimator_.steps[0][1]

On applique l'algorithme sur nos données

In [33]:
data_vectorized = tf_vectorizer.transform(x_train_text)
lda_output = best_lda_model.transform(data_vectorized)

In [34]:
tags_by_topics_lda = buildTagByTopicsMatrix(best_lda_model, lda_output, y_train)

### Evaluation

In [78]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", best_lda_model.score(data_vectorized))
# A lower perplexity score indicates better generalization performance.
print("Perplexity : ",best_lda_model.perplexity(data_vectorized))

Log Likelihood:  -10119260.0448
Perplexity :  1154.02049568


In [48]:
y_predicted_lda = getPostsTagsPrediction(x_test, 5, tags_by_topics_lda, tf_vectorizer, best_lda_model, lda_output)
predictionAccuracy(y_predicted_lda,y_test)

Prediction accuracy: 28.64 % 


### Evaluation de la prédiction de notre sample de validation

In [49]:
x_validation = df_validation[['TITLE_P','BODY_P']]
y_validation = df_validation['TAGS_P']
y_validation = y_validation.to_frame()

In [50]:
y_predicted = getPostsTagsPrediction(x_validation, 5,tags_by_topics_lda, tf_vectorizer, best_lda_model, lda_output )
predictionAccuracy(y_predicted, y_validation)

Prediction accuracy: 25.18 % 


## 4.2 Tuning manuel

### Apprentissage

In [82]:
def fit_transform_LDA(train_data, target_data, min_df, max_df, num_topics) :
    print("[min_df:{:d} max_df:{:.2f} num_topics:{}]"
          .format(min_df, max_df, num_topics), end=" ", flush=True)
    tf_vectorizer = CountVectorizer(min_df=min_df, max_df=max_df)
    data_vectorized = tf_vectorizer.fit_transform(train_data['TITLE_P'] + ' ' + train_data['BODY_P'])
    
    lda_model = LatentDirichletAllocation(n_components=num_topics, max_iter=5, learning_method='online')
    lda_output = lda_model.fit_transform(data_vectorized)
    print("Log Likelihood: {:.2f} - Perplexity: {:.2f}".format(lda_model.score(data_vectorized), 
                                                               lda_model.perplexity(data_vectorized)), 
          end=" ", flush=True)
    tags_by_topics_matrix = buildTagByTopicsMatrix(lda_model, lda_output, target_data)
    return lda_output, lda_model, tf_vectorizer, tags_by_topics_matrix

On fait varier min_df, max_df et num_topics.

### Evaluation

In [80]:
min_df_values = [5, 10, 20]
max_df_values = [0.75, 0.85, 0.95]
num_topics = [10, 20, 30]

In [84]:
for n_topics in num_topics :
    for min_df in min_df_values :
        for max_df in max_df_values :
            lda_output_m, lda_model_m, tf_vectorizer_m, tags_by_topics_m = fit_transform_LDA(x_train, y_train, min_df, max_df, n_topics)
            y_predicted_lda = getPostsTagsPrediction(x_test, 5, tags_by_topics_m, tf_vectorizer_m, lda_model_m, lda_output_m)
            predictionAccuracy(y_predicted_lda,y_test)

[min_df:5 max_df:0.75 num_topics:10] Log Likelihood: -10260395.55 - Perplexity: 1273.28 Prediction accuracy: 21.24 % 
[min_df:5 max_df:0.85 num_topics:10] Log Likelihood: -10258437.21 - Perplexity: 1271.54 Prediction accuracy: 22.61 % 
[min_df:5 max_df:0.95 num_topics:10] Log Likelihood: -10230798.19 - Perplexity: 1247.29 Prediction accuracy: 23.36 % 
[min_df:10 max_df:0.75 num_topics:10] Log Likelihood: -9617426.10 - Perplexity: 1056.81 Prediction accuracy: 22.37 % 
[min_df:10 max_df:0.85 num_topics:10] Log Likelihood: -9588039.57 - Perplexity: 1034.56 Prediction accuracy: 21.26 % 
[min_df:10 max_df:0.95 num_topics:10] Log Likelihood: -9597821.80 - Perplexity: 1041.91 Prediction accuracy: 22.08 % 
[min_df:20 max_df:0.75 num_topics:10] Log Likelihood: -8909896.94 - Perplexity: 863.34 

KeyboardInterrupt: 

Manuellement nous obtenons le meilleur score avec 30 topics, un min_df à 20 et un max_df à 0.95.<br/>
Mais l'accuracy reste plus faible que le GridSearchCV et nous selectionnerons donc le modèle tunné par le GridSearch.

### Evaluation de la prédiction de notre sample de validation

In [89]:
lda_output_m, lda_model_m, tf_vectorizer_m, tags_by_topics_m = fit_transform_LDA(x_train, y_train, 
                                                                                 5, 0.95, 20)

[min_df:5 max_df:0.95 num_topics:20] Log Likelihood: -10108997.49 - Perplexity: 1145.80 

In [90]:
y_predicted = getPostsTagsPrediction(x_validation, 5,tags_by_topics_m, tf_vectorizer_m, lda_model_m, lda_output_m )
predictionAccuracy(y_predicted, y_validation)

Prediction accuracy: 0.75 % 


## 4.3 Ajustement du nombre de tags

In [95]:
num_tags = [3, 4, 5, 6, 7, 8]
for n in num_tags :
    print("{} tags :".format(n), end=" ", flush=True)
    y_predicted = getPostsTagsPrediction(x_validation, n,tags_by_topics_lda, tf_vectorizer, best_lda_model, lda_output )
    predictionAccuracy(y_predicted, y_validation)

3 tags : Prediction accuracy: 19.83 % 
4 tags : Prediction accuracy: 22.50 % 
5 tags : Prediction accuracy: 25.18 % 
6 tags : Prediction accuracy: 27.25 % 
7 tags : Prediction accuracy: 29.03 % 
8 tags : Prediction accuracy: 30.32 % 


=> Nous remarquons que plus nous augmenter le nombre de tags, meilleure est la prédiction. <br/>
=> Cependant, il faut rester surun nombre raisonnable de tags et nous conseiller de partir sur 6 tags.

## 4.4 Analyse de la modélisation

### Topics / Words

Affichage de la matrice topics / words.

In [96]:
getTopicsWordsMatrix(best_lda_model, tf_vectorizer.get_feature_names())

,aa,aaa,aaaa,aaaaa,aac,aapt,aar,ab,abandon,abbrevi,...,zookeep,zoom,zuul,zygoteinit,zz,zza,zzb,zzd,zzz,µs
Topic0,10.098274,2.288014,2.860817,0.050000,0.052178,0.050000,0.050000,2.070085,7.182516,1.712503,...,0.050000,0.050390,13.503588,0.050000,0.050000,0.05,0.05,0.05,0.050000,0.050000
Topic1,0.056834,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.922529,0.050000,0.050000,...,0.050000,8.957463,0.050000,0.050000,0.050000,0.05,0.05,0.05,0.050000,0.050000
Topic2,1.119772,1.788745,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050345,0.050000,0.050000,0.050000,0.05,0.05,0.05,4.864339,0.050000
Topic3,4.445646,0.050000,0.050000,0.050000,12.118017,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05,0.05,0.05,0.050000,0.050000
Topic4,3.682235,0.050000,0.050000,1.267049,5.246112,1.379417,0.050000,41.668319,0.050000,0.050000,...,11.420927,0.535132,0.050000,0.050000,0.050000,0.05,0.05,0.05,0.050000,49.036386
Topic5,0.050000,2.195132,0.200023,0.050000,0.050000,15.614266,0.050041,20.591882,3.719576,2.192743,...,0.050000,3.961148,0.050000,0.050000,0.050000,0.05,0.05,0.05,4.359932,0.050000
Topic6,0.755392,0.050000,0.050000,0.485173,0.050000,0.050384,89.039271,2.169274,0.050000,0.050000,...,0.050000,0.050000,0.050000,308.049982,3.730596,41.05,13.05,16.05,1.055669,0.050000
Topic7,13.316234,10.777035,0.050000,0.050000,3.783693,0.155933,0.050000,8.789495,0.480437,0.050000,...,0.051025,4.344899,0.050000,0.050018,0.050000,0.05,0.05,0.05,0.050000,0.050000
Topic8,0.050000,8.476138,0.050000,1.604337,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,88.098001,0.050000,0.050000,0.050000,0.05,0.05,0.05,0.050000,0.050000
Topic9,0.052269,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,1.316010,0.050000,0.050000,...,0.050000,0.072445,0.050000,0.050000,0.484605,0.05,0.05,0.05,0.050000,0.050000


In [98]:
showTopicsTopWords(best_lda_model, tf_vectorizer.get_feature_names(), 12)

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11
Topic 0,use,file,app,project,user,work,get,net,error,tri,http,api
Topic 1,in,python,lib,import,tf,np,from,size,model,data,the,name
Topic 2,function,data,log,request,server,var,http,error,in,query,json,post
Topic 3,let,use,list,self,data,the,swift,video,like,func,if,function
Topic 4,array,use,string,would,time,like,object,address,valu,one,ms,new
Topic 5,build,run,test,command,use,error,the,version,debug,xcode,task,bin
Topic 6,android,java,com,at,gradle,app,compile,google,support,internal,org,activitythread
Topic 7,public,new,string,void,class,name,private,the,if,file,this,return
Topic 8,div,class,px,button,width,text,id,style,height,html,color,li
Topic 9,this,git,state,branch,false,commit,view,text,work,chang,pull,repositori


Nous pouvons interpréter certains topics : <br/>
- le sujet 1 : Tableau HTML
- Le sujet 7 : Androïd 
- Le sujet 9 : Requêtes HTTP 
- Le sujet 15 : Microsoft / .Net
- Le sujet 16 : Java / Apache
- Le sujet 17 : Python
- Le sujet 19 : System / Installation
...

###  Docs / Topics

Voyons maintenant le topic associé à quelques documents.

In [100]:
showTopicsTopDocs(best_lda_model.components_, lda_output, tf_vectorizer.get_feature_names(), x_train, y_train, 10, 2)

Topic 0 : use file app project user work get net error tri
Doc 1949  Title: Edit & Continue doesn't work - Tags: ['visual-studio', 'visual-studio-2015', 'visual-studio-debugging']
Doc 5554  Title: ASP.net core web api: Using Facebook/Google OAuth access tok - Tags: ['c#', 'angular', 'authentication', 'google-oauth', 'asp.net-core-webapi']

Topic 1 : in python lib import tf np from size model data
Doc 1142  Title: Tensorflow ResourceExhaustedError after first batch - Tags: ['python', 'python-3.x', 'tensorflow']
Doc 3059  Title: Edit tensorflow inceptionV3 retraining-example.py for multip - Tags: ['python', 'python-3.x', 'machine-learning', 'tensorflow']

Topic 2 : function data log request server var http error in query
Doc 693  Title: INSERT operation using Node.js and PostgreSQL doesn't work - Tags: ['sql', 'node.js', 'postgresql', 'async-await']
Doc 4615  Title: XMLHttpRequest cannot load resource - Tags: ['php', 'jquery', 'ajax', 'heroku', 'cors']

Topic 3 : let use list self data t

In [101]:
getDocsTopicsMatrix(x_train, best_lda_model, lda_output).head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Dominant_Topic
Doc0,0.1999,0.0003,0.1005,0.0003,0.0003,0.0003,0.0003,0.0003,0.1728,0.0003,...,0.0003,0.1345,0.0003,0.3873,0.0003,0.0003,0.0003,0.0003,0.0003,14
Doc1,0.0003,0.0003,0.7595,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,...,0.0003,0.2359,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,2
Doc2,0.9849,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,...,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0
Doc3,0.1597,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.3704,0.0003,0.0003,...,0.0003,0.0003,0.0003,0.4158,0.0500,0.0003,0.0003,0.0003,0.0003,14
Doc4,0.0003,0.0003,0.0003,0.0003,0.3964,0.0003,0.0003,0.0003,0.0003,0.0003,...,0.0003,0.0003,0.5973,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,13
Doc5,0.5811,0.0002,0.0002,0.1859,0.0002,0.0002,0.0826,0.0002,0.1113,0.0002,...,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0359,0
Doc6,0.0002,0.0002,0.0677,0.0002,0.0002,0.0002,0.0002,0.0504,0.5154,0.0002,...,0.0002,0.0002,0.0002,0.2922,0.0002,0.0002,0.0002,0.0002,0.0002,8
Doc7,0.0001,0.0001,0.0001,0.1139,0.0001,0.6305,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.2414,0.0001,0.0001,0.0132,0.0001,0.0001,0.0001,5
Doc8,0.0019,0.0019,0.0019,0.0019,0.0019,0.5310,0.0019,0.0019,0.0019,0.0019,...,0.0019,0.0019,0.0019,0.0019,0.0019,0.0019,0.4344,0.0019,0.0019,5
Doc9,0.0009,0.0009,0.0009,0.0009,0.0009,0.2774,0.0009,0.0009,0.0009,0.0009,...,0.0009,0.2389,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.4677,19


# 5. Topic modeling avec NMF

Voyons maintenant la méthode NMF (Factorisation par matrices non négatives) pour prédire nos tags.

## 5.1 Tuning manuel

### Apprentissage

In [122]:
def fit_transform_NMF(train_data, target_data, min_df, max_df, num_topics) :
    print("[min_df:{:d} max_df:{:.2f} num_topics:{}]"
          .format(min_df, max_df, num_topics), end=" ", flush=True)
    tfidf_vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    tfidf_vectorized = tfidf_vectorizer.fit_transform(train_data['TITLE_P'] + ' ' + train_data['BODY_P'])
    nmf_model = NMF(n_components=num_topics, random_state=1, alpha=.1, l1_ratio=.5)
    nmf_output = nmf_model.fit_transform(tfidf_vectorized)
    tags_by_topics_matrix = buildTagByTopicsMatrix(nmf_model, nmf_output, target_data)
    return nmf_output, nmf_model, tfidf_vectorizer, tags_by_topics_matrix

### Evaluation

In [123]:
for n_topics in num_topics :
    for min_df in min_df_values :
        for max_df in max_df_values :
            nmf_output_m, nmf_model_m, tfidf_vectorizer_m, tags_by_topics_m = fit_transform_NMF(x_train, y_train, min_df, max_df, n_topics)
            y_predicted_nmf = getPostsTagsPrediction(x_test, 5, tags_by_topics_m, tfidf_vectorizer_m, nmf_model_m, nmf_output_m)
            predictionAccuracy(y_predicted_nmf,y_test)

[min_df:5 max_df:0.75 num_topics:10] Prediction accuracy: 26.20 % 
[min_df:5 max_df:0.85 num_topics:10] Prediction accuracy: 26.20 % 
[min_df:5 max_df:0.95 num_topics:10] Prediction accuracy: 26.20 % 
[min_df:10 max_df:0.75 num_topics:10] Prediction accuracy: 25.83 % 
[min_df:10 max_df:0.85 num_topics:10] Prediction accuracy: 25.83 % 
[min_df:10 max_df:0.95 num_topics:10] Prediction accuracy: 25.83 % 
[min_df:20 max_df:0.75 num_topics:10] Prediction accuracy: 27.48 % 
[min_df:20 max_df:0.85 num_topics:10] Prediction accuracy: 27.48 % 
[min_df:20 max_df:0.95 num_topics:10] Prediction accuracy: 27.48 % 
[min_df:5 max_df:0.75 num_topics:20] Prediction accuracy: 31.36 % 
[min_df:5 max_df:0.85 num_topics:20] Prediction accuracy: 31.36 % 
[min_df:5 max_df:0.95 num_topics:20] Prediction accuracy: 31.36 % 
[min_df:10 max_df:0.75 num_topics:20] Prediction accuracy: 30.37 % 
[min_df:10 max_df:0.85 num_topics:20] Prediction accuracy: 30.37 % 
[min_df:10 max_df:0.95 num_topics:20] 

KeyboardInterrupt: 

### Evaluation de la prédiction de notre sample de validation

In [124]:
nmf_output, best_nmf_model, tfidf_vectorizer, tags_by_topics_nmf = fit_transform_NMF(x_train, y_train, 5, 0.95, 20)

[min_df:5 max_df:0.95 num_topics:20] 

In [125]:
y_predicted = getPostsTagsPrediction(x_validation, 5,tags_by_topics_nmf, tfidf_vectorizer, best_nmf_model, nmf_output)
predictionAccuracy(y_predicted, y_validation)

Prediction accuracy: 27.25 % 


## 5.2 Ajustement du nombre de tags

In [129]:
num_tags = [3, 4, 5, 6, 7, 8]
for n in num_tags :
    print("{} tags :".format(n), end=" ", flush=True)
    y_predicted = getPostsTagsPrediction(x_validation, n, tags_by_topics_nmf, tfidf_vectorizer, best_nmf_model, nmf_output)
    predictionAccuracy(y_predicted, y_validation)

3 tags : Prediction accuracy: 23.02 % 
4 tags : Prediction accuracy: 25.35 % 
5 tags : 

KeyboardInterrupt: 

## 5.3 Analyse

### Topics / Words

In [130]:
showTopicsTopWords(best_nmf_model, tfidf_vectorizer.get_feature_names(), 12)

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11
Topic 0,use,would,like,test,code,way,time,one,work,set,type,want
Topic 1,android,com,layout,support,app,compile,google,intent,studio,view,build,gradl
Topic 2,std,int,const,struct,type,include,cout,template,void,compil,typename,vector
Topic 3,div,class,button,form,span,id,input,html,label,ng,this,col
Topic 4,java,at,org,lang,com,internal,activitythread,apache,util,os,gradle,zygoteinit
Topic 5,angular,js,import,node,modul,ts,app,webpack,from,npm,cli,ng
Topic 6,px,width,height,color,background,style,text,border,css,imag,font,margin
Topic 7,list,item,li,element,map,sort,id,items,filter,in,stream,ul
Topic 8,net,project,studio,visual,core,asp,instal,version,microsoft,packag,nuget,vs
Topic 9,file,upload,folder,directori,path,git,json,open,read,files,download,js
